# Google Maps Distance and Duration Calculator

## NEED TO BE REVISED AND ORGANIZED

## TLDR:

This Python script uses the Google Maps Distance Matrix API to calculate the distance and travel time between two sets of GPS coordinates. The data is read from a CSV file, the calculations are performed, and the results are written back into the dataset.

## Author:

Tiago Tamagusko (tamagusko@gmail.com)  
Version: 0.1 beta  
Date: 2023-06-23

## Description:

The script takes a dataset containing route data, specifically origin and destination coordinates, and uses the Google Maps Distance Matrix API to calculate the travel time and distance between these coordinates. The calculated values are then added back to the dataset. If the API returns an error, the error is logged, and the processing continues to the next row.

## Functionality:

1. `load_data()`: Loads data from a CSV file uploaded from your local machine.

2. `calculate_distance_and_time(df, api_key)`: Calculates distance and travel time between origin and destination coordinates using the Google Maps Distance Matrix API. The calculated values are written back into the DataFrame. Any errors encountered during API calls are printed.

3. `save_data(df, file_name)`: Saves the DataFrame, including the calculated distance and travel time, to a new CSV file and downloads it to your local machine.

## Usage:

- Upload a CSV file with route data, specifically containing 'route_id', 'origin_latitude', 'origin_longitude', 'dest_latitude', and 'dest_longitude' columns.
- Call `load_data()` function to load the CSV file into a DataFrame.
- Call `calculate_distance_and_time(df, api_key)` function to calculate the distance and travel time for each route. Replace 'api_key' with your Google Maps Distance Matrix API key.
- Call `save_data(df, file_name)` function to save the updated DataFrame to a CSV file. Replace 'file_name' with the desired name for the output CSV file.

## Requirements:

- pandas
- requests
- Google Maps Distance Matrix API key

In [ ]:
import io
import requests
import pandas as pd
from google.colab import files


def load_data():
    """Loads data from a CSV file uploaded from the user's local machine.

    Returns:
        df: DataFrame containing the data from the uploaded CSV file.
    """
    uploaded_file = files.upload()
    file_name = list(uploaded_file.keys())[0]
    df = pd.read_csv(io.BytesIO(uploaded_file[file_name]))
    return df


def calculate_distance_and_time(df, api_key):
    """Calculates distance and travel time between two coordinates using the Google Maps Distance Matrix API.
    Use bycicle as travel mode.

    Args:
        df: DataFrame containing the dataset.
        api_key: String representing the API key for the Google Maps Distance Matrix API.

    Returns:
        df: DataFrame after the distance and travel time calculations.
    """
    base_url = "https://maps.googleapis.com/maps/api/distancematrix/json?"

    for index, row in df.iterrows():
        orig_coords = f"{row['origin_latitude']},{row['origin_longitude']}"
        dest_coords = f"{row['dest_latitude']},{row['dest_longitude']}"

        req_url = f"{base_url}origins={orig_coords}&destinations={dest_coords}&mode=bicycling&key={api_key}"
        response = requests.get(req_url).json()

        if response['rows'][0]['elements'][0]['status'] == 'OK':
            df.at[index, 'duration'] = response['rows'][0]['elements'][0]['duration']['value'] / 60
            df.at[index, 'distance'] = response['rows'][0]['elements'][0]['distance']['value'] / 1000
            df.at[index, 'route_type'] = 'google'
        else:
            print(f"Error calculating distance for route_id {row['route_id']}")

    return df



def save_data(df, file_name):
    """Saves the DataFrame to a CSV file and downloads it to the user's local machine.

    Args:
        df: DataFrame containing the dataset.
        file_name: String representing the name of the file.
    """
    df.to_csv(file_name, index=False)
    files.download(file_name)


In [ ]:
df = load_data()

Saving 100routesResultComplete.csv to 100routesResultComplete.csv


In [ ]:
# Clean data
def drop_columns(df, columns_to_drop):
    return df.drop(columns=columns_to_drop, errors='ignore')


def drop_duplicates(df):
    return df.drop_duplicates()


def rename_columns(df, columns):
    return df.rename(columns=columns)



In [ ]:
df = load_data()
columns_to_drop = ['error_description', 'test_code', 'test_error_description', 'duration_min', 'route_factor']
df = drop_columns(df, columns_to_drop)
df = drop_duplicates(df)

columns = {'duration': 'duration_min'}

df = rename_columns(df, columns)

Saving updated_routes.csv to updated_routes (5).csv


In [ ]:
save_data(df, "100routesResultGoogle.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = load_data()

Saving 100routesResultComplete.csv to 100routesResultComplete (1).csv


In [ ]:
df.head()
columns_to_drop = ['error_description', 'test_code', 'test_error_description']
df = drop_columns(df, columns_to_drop)
save_data(df, "100routesResultComplete.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def main():
    df = load_data()
    api_key = "YOUR API KEY HERE"
    df = calculate_distance_and_time(df, api_key)
    save_data(df, "updated_routes.csv")


if __name__ == "__main__":
    main()